In [1]:
; cl-waffeを読み込む
(load "../../cl-waffe.asd")
(ql:quickload :cl-waffe :silent t)
(use-package :cl-waffe)
(use-package :cl-waffe.nn)

T

(:CL-WAFFE)

T

T

Unable to find framework CUDA


### Seq2Seq

(まだ何も書いてないです)

In [2]:
(defmodel Encoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1)))
  :forward ((x)
            (with-calling-layers x
                (embedding x)
                (layer x))))

(defmodel Decoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1))
               (h2l       (linearlayer hidden-size vocab-size)))
  
  :forward ((encoder-state y)
            (let* ((ye (call (self embedding) y))
                   (hs (call (self layer) ye encoder-state))
                   (h-output (call (self h2l) hs)))
                  (list h-output hs))))

(defmodel Seq2Seq (vocab-size embedding-dim input-size)
  :parameters ((encoder (Encoder vocab-size embedding-dim input-size))
               (decoder (Decoder vocab-size embedding-dim input-size)))  
  :forward ((x y)
            (let ((x-state (call (self encoder) x))
                  (y1 (!zeros (!shape y))))
                 (setq y1 (setf (!aref y1 t) (!aref y '(1 t))))
                 (call (self decoder) x-state y1))))

NIL

NIL

NIL

SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::ENCODER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::ENCODER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::ENCODER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: undefined type: COMMON-LISP-USER::ENCODER
SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMM

In [3]:
(deftrainer Seq2SeqTrainer (vocab-size embedding-dim hidden-size)
  :model (Seq2Seq vocab-size embedding-dim hidden-size)
  :optimizer cl-waffe.optimizers:Adam
  :optimizer (:lr 1e-4)
  :step-model ((x y)
               (zero-grad)
               (let* ((outs (call (self model) x y))
                      (out (softmax-cross-entropy (car outs) y)))
                     (backward out)
                     (update)
                     out))
  :predict ((x) (call (model) x)))

NIL

SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::SEQ2SEQTRAINER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::SEQ2SEQTRAINER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.
SB-INT:SIMPLE-STYLE-WARNING: Cannot find type for specializer
                             COMMON-LISP-USER::SEQ2SEQTRAINER when executing
                             SB-PCL:SPECIALIZER-TYPE-SPECIFIER for a
                             STANDARD-METHOD of a STANDARD-GENERIC-FUNCTION.


In [4]:
(setq model (seq2seqtrainer 10 256 256))

<Trainer: SEQ2SEQTRAINER()>

SIMPLE-WARNING: undefined variable: CL-WAFFE:MODEL


In [5]:
(setq x (!ones `(2 10)))
(setq y (!ones `(2 10)))

(time (dotimes (i 10) (print (step-model model x y))))

#Const(((1.0 1.0 ~ 1.0 1.0)
        (1.0 1.0 ~ 1.0 1.0)) :mgl t :shape (2 10) :backward NIL)

#Const(((1.0 1.0 ~ 1.0 1.0)
        (1.0 1.0 ~ 1.0 1.0)) :mgl t :shape (2 10) :backward NIL)

NIL

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Y
SIMPLE-WARNING: undefined variable: CL-WAFFE:MODEL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Y

#Const(2.302652) 
#Const(2.2956505) 
#Const(2.2856839) 
#Const(2.268782) 
#Const(2.238607) 
#Const(2.1823976) 
#Const(2.0743089) 
#Const(1.8632463) 
#Const(1.466358) 
#Const(0.8976947) 
Evaluation took:
  8.083 seconds of real time
  8.343875 seconds of total run time (6.913872 user, 1.430003 system)
  [ Run times consist of 0.628 seconds GC time, and 7.716 seconds non-GC time. ]
  103.23% CPU
  220 lambdas converted
  18,623,367,250 processor cycles
  10,929,126,512 bytes consed
  
